# Setup of the AnnData object
**Author:** [Severin Dicks](https://github.com/Intron7) (IBSM Freiburg)

This notebook is just downloader and sets up the AnnData object (https://anndata.readthedocs.io/en/latest/index.html) we will be working with. In this example workflow we'll be looking at a dataset of ca. 90000 cells from lungcancer patients published by [Quin et al., Cell Research 2020](https://www.nature.com/articles/s41422-020-0355-0).
In the Pearson Residuals example we'll use a dataset of 200000 brain cells from [Nvidia](https://github.com/clara-parabricks/rapids-single-cell-examples/blob/master/notebooks/1M_brain_cpu_analysis.ipynb).



In [1]:
import gdown
import os
import wget
import scanpy as sc
os.makedirs("./h5",exist_ok=True)

In [2]:
url = 'https://drive.google.com/file/d/1eoK0m2ML1uNLc80L6yBuPrkJqsDF-QWj/view?usp=sharing'
output = './h5/adata.raw.h5ad'
gdown.download(url, output, quiet=True, fuzzy=True)

'./h5/adata.raw.h5ad'

In [3]:
wget.download('https://rapids-single-cell-examples.s3.us-east-2.amazonaws.com/1M_brain_cells_10X.sparse.h5ad',
              "h5/nvidia_1.3M.h5ad")
adata = sc.read("h5/nvidia_1.3M.h5ad")
adata.var_names_make_unique()
adata = adata[:200000,:]
adata.write("h5/200000.h5ad")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


TypeError: write() missing 1 required positional argument: 'adata'

In [6]:
adata = sc.read("h5/nvidia_1.3M.h5ad")
adata = adata[:500000,:]
adata.write("h5/500000.h5ad")

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [ ]:
adata = sc.read("h5/nvidia_1.3M.h5ad")
adata = adata[:200000,:]
adata.var_names_make_unique()
adata.write("h5/200000.h5ad")